In [1]:
from common import *

DAY = 10
lines = get_input_lines(DAY)

In [2]:
lines = '''.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...'''.splitlines()

In [3]:
from functools import lru_cache
from typing import Callable, Iterable, Self


class Map:
    def __init__(self, h: int, w: int, value: int | str = -1) -> None:
        self.w = w
        self.h = h
        self.map = self.create(h, w, value)
        self.show_remap: dict[int | str, int | str] = {-1: '.'}
        self.get_nearest_points = lru_cache(maxsize=None)(self.get_nearest_points)

    @classmethod
    def from_object(cls, lines: list[str]) -> Self:
        h = len(lines)
        w = len(lines[0])
        map = cls(h, w)
        for y, line in enumerate(lines):
            for x, value in enumerate(line):
                map.map[y][x] = value
        return map

    def create(self, y: int, x: int, value: int | str = -1) -> list[list[int | str]]:
        line = [value] * x
        return [line[:] for _ in range(y)]

    def show(self, width: int = 2) -> None:
        for line in self.map:
            for value in line:
                value = self.show_remap.get(value, value)
                print(f'{value:>{width}} ', end='')
            print()

    def find_value(self, value: int | str) -> tuple[int, int]:
        for y, line in enumerate(self.map):
            for x, d in enumerate(line):
                if d == value:
                    return y, x
        raise ValueError('Not found')

    def normalize_point(self, y: int, x: int) -> tuple[int, int] | None:
        if not (0 <= y < self.h) or not (0 <= x < self.w):
            return None
        return y, x

    def get_nearest_points(self, y: int, x: int) -> list[tuple[int, int]]:
        result = []
        for ydiff, xdiff in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_coords = self.normalize_point(y + ydiff, x + xdiff)
            if new_coords is not None:
                result.append(new_coords)
        return result

    def get_nearest_points_by_value(self, y: int, x: int, value: int) -> Iterable[tuple[int, int, int | str]]:
        for new_y, new_x in self.get_nearest_points(y, x):
            new_value = self.map[new_y][new_x]
            if new_value == value:
                yield new_y, new_x, new_value

    def calc_inc_step(self, condition: Callable | None = None) -> bool:
        def target_filter(y, x, value, new_y, new_x, new_value):
            if condition is not None and not condition(y, x, value, new_y, new_x, new_value):
                return False
            return new_value == -1 or new_value > value + 1

        def target_action(y, x, value, new_y, new_x, new_value):
            self.map[new_y][new_x] = value + 1

        return self.calc_step(
            point_filter=lambda y, x, value: value != -1,
            target_filter=target_filter,
            target_action=target_action,
        )

    def calc_step(
        self,
        point_filter: Callable | None = None,
        target_filter: Callable | None = None,
        target_action: Callable | None = None,
        target_value: int | str | None = None,
    ) -> bool:
        any_change = False
        for y, line in enumerate(self.map):
            for x, value in enumerate(line):
                if point_filter is not None and not point_filter(y, x, value):
                    continue
                for new_y, new_x in self.get_nearest_points(y, x):
                    new_value = self.map[new_y][new_x]
                    if target_filter is not None and not target_filter(y, x, value, new_y, new_x, new_value):
                        continue
                    if target_value is not None:
                        self.map[new_y][new_x] = target_value
                    if target_action is not None:
                        target_action(y, x, value, new_y, new_x, new_value)
                    any_change = True
        return any_change

In [4]:
directions = {
    (0, -1): '-J7S',
    (0, 1): '-LFS',
    (-1, 0): '|LJS',
    (1, 0): '|7FS',
}
directions_inv = {(k[0] * -1, k[1] * -1): v for k, v in directions.items()}

def is_connected(y: int, x: int, value: int, new_y: int, new_x: int, new_value: int) -> bool:
    diff = (new_y - y, new_x - x)
    return lines[y][x] in directions[diff] and lines[new_y][new_x] in directions_inv[diff]

In [5]:
dmap = Map(len(lines), len(lines[0]))
y, x = Map.from_object(lines).find_value('S')

dmap.map[y][x] = 0
while dmap.calc_inc_step(condition=is_connected):
    pass

dmap.show()

 . 59 60 61 62 63 64 69 68 61 60 51 50 43 42 41  .  .  .  . 
 . 58 47 46 45 44 65 70 67 62 59 52 49 44 39 40  .  .  .  . 
 . 57 48  . 42 43 66 69 66 63 58 53 48 45 38 37  .  .  .  . 
55 56 49 50 41 40 67 68 65 64 57 54 47 46  . 36 35 34  .  . 
54 53 52 51  . 39 38  .  .  . 56 55  0  1 20 21 22 33 32  . 
 .  .  .  . 35 36 37  .  . 14 13  2  1  2 19 18 23 24 31 30 
 .  .  .  . 34 33  . 19 18 15 12  3  4  3  . 17 16 25 26 29 
 .  .  .  .  . 32 21 20 17 16 11  6  5  4  9 10 15  . 27 28 
 .  .  .  . 30 31 22 23 24  . 10  7  .  5  8 11 14  .  .  . 
 .  .  .  . 29 28 27 26 25  .  9  8  .  6  7 12 13  .  .  . 


In [6]:
result = max(max(line) for line in dmap.map)
# assert result == 6867

In [7]:
def change_coord(x):
    return x * 2 + 2

pmap = Map(change_coord(dmap.h), change_coord(dmap.w))

max_d = max(max(line) for line in dmap.map)
start_y, start_x = dmap.find_value(max_d)
pmap.map[change_coord(start_y)][change_coord(start_x)] = max_d

for y, x, _ in dmap.get_nearest_points_by_value(start_y, start_x, max_d - 1):
    pmap.map[change_coord(start_y) + y - start_y][change_coord(start_x) + x - start_x] = dmap.map[y][x]
    pmap.map[change_coord(y)][change_coord(x)] = dmap.map[y][x]
    while True:
        num = dmap.map[y][x]
        for new_y, new_x, new_num in dmap.get_nearest_points_by_value(y, x, num - 1):
            pmap.map[change_coord(new_y)][change_coord(new_x)] = new_num
            pmap.map[change_coord(y) + new_y - y][change_coord(x) + new_x - x] = new_num
            y, x = new_y, new_x
            break
        else:
            break

pmap.show()

 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  . 59 59 60 60 61 61 62 62 63 63 64  . 69 68 68  . 61 60 60  . 51 50 50  . 43 42 42 41 41  .  .  .  .  .  .  .  .  . 
 .  .  .  . 58  .  .  .  .  .  .  .  .  . 64  . 69  . 67  . 61  . 59  . 51  . 49  . 43  .  .  . 40  .  .  .  .  .  .  .  .  . 
 .  .  .  . 58  . 47 46 46 45 45 44 44  . 65  . 70  . 67  . 62  . 59  . 52  . 49  . 44  . 39 39 40  .  .  .  .  .  .  .  .  . 
 .  .  .  . 57  . 47  .  .  .  .  . 43  . 65  . 69  . 66  . 62  . 58  . 52  . 48  . 44  . 38  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  . 57  . 48  .  .  . 42 42 43  . 66  . 69  . 66  . 63  . 58  . 53  . 48  . 45  . 38 37 37  .  .  .  .  .  .  .  .  . 
 .  .  .  . 56  . 48  .  .  . 41  .  .  . 66  . 68  . 65  . 63  . 57  . 53  . 47  . 45  .  .  . 36  .  .  .  . 

In [8]:
pmap.map[0][0] = 'O'

while pmap.calc_step(
    point_filter=lambda y, x, value: value == 'O',
    target_filter=lambda y, x, value, new_y, new_x, new_value: new_value == -1,
    target_value='O',
):
    pass

pmap.show()

 O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
 O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O  O 
 O  O  O  O 59 59 60 60 61 61 62 62 63 63 64  O 69 68 68  O 61 60 60  O 51 50 50  O 43 42 42 41 41  O  O  O  O  O  O  O  O  O 
 O  O  O  O 58  .  .  .  .  .  .  .  .  . 64  O 69  . 67  O 61  . 59  O 51  . 49  O 43  .  .  . 40  O  O  O  O  O  O  O  O  O 
 O  O  O  O 58  . 47 46 46 45 45 44 44  . 65  O 70  . 67  O 62  . 59  O 52  . 49  O 44  . 39 39 40  O  O  O  O  O  O  O  O  O 
 O  O  O  O 57  . 47  O  O  O  O  O 43  . 65  O 69  . 66  O 62  . 58  O 52  . 48  O 44  . 38  O  O  O  O  O  O  O  O  O  O  O 
 O  O  O  O 57  . 48  O  O  O 42 42 43  . 66  O 69  . 66  O 63  . 58  O 53  . 48  O 45  . 38 37 37  O  O  O  O  O  O  O  O  O 
 O  O  O  O 56  . 48  O  O  O 41  .  .  . 66  O 68  . 65  O 63  . 57  O 53  . 47  O 45  .  .  . 36  O  O  O  O 

In [9]:
result = 0
for y, line in enumerate(pmap.map):
    for x, value in enumerate(line):
        if value == -1 and x % 2 == 0 and y % 2 == 0:
            result += 1
# assert result == 595